# Imperative Workflow


## Why an “imperative” API?

The default WorkGraph engines ask you to **declare** every task and dependency *before* execution.
The *imperative* engine in **aiida‑workgraph** lets you write an ordinary Python `async` function instead.  Each time you call a task inside that function the engine immediately schedules it and updates the WorkGraph on‑the‑fly.

*Benefits*

* **Natural control‑flow** – use native `while`, `for`, `if/else`, exceptions.
* **Incremental graphs** – dependencies are inferred automatically.
* **Rapid prototyping** – no context variables or DSL constructs required.

If you need full static inspection before running, the declarative API is still available; you can even mix both styles in one project.

## The simplest flow


In [ ]:
from aiida_workgraph import task
from aiida_workgraph.engine.imperative.imperative import WorkGraphImperativeEngine
from aiida.engine import run, submit
from aiida import orm, load_profile

load_profile()

@task.pythonjob()
def add(x, y):
    return x + y

@task.pythonjob()
def multiply(x, y):
    return x * y

async def add_multiply(x, y):
    a = add(x, y)
    b = multiply(1, a.result)   # chain directly on the result
    return {"sum": a.result, "product": b.result}

results = run(
    WorkGraphImperativeEngine,
    inputs={"workgraph_data": {
        "name": "add_multiply",
        "flow": add_multiply,
        "function_inputs": {"x": orm.Int(3), "y": orm.Int(4)}
    }},
)
print(results)


### What happens under the hood?

1. The engine creates a fresh WorkGraph called **add\_multiply**.
2. `add()` is scheduled; its output node is stored as `a`.
3. `multiply()` is scheduled as soon as its dependency (`a.result`) is ready.
4. When the async function returns, the engine marks the WorkGraph **FINISHED** and returns `results`.

> **Tip** You use the GUI to inspect the evolving DAG.

---

## Conditional branches (`if/else`)

In [ ]:
from aiida_workgraph.engine.imperative.imperative import wait_for

@task.pythonjob()
def add(x, y):
    import time; time.sleep(2)  # simulate work
    return x + y

@task.pythonjob()
def multiply(x, y):
    import time; time.sleep(2)
    return x * y

async def add_then_branch(x, y):
    a = add(x, y)
    await wait_for(a)           # don\'t read the result too early

    if a.result.value > 10:
        m = multiply(1, a.result)
    else:
        m = multiply(-1, a.result)

    return {"sum": a, "multiply": m}

results = run(
    WorkGraphImperativeEngine,
    inputs={"workgraph_data": {
        "name": "add_multiply",
        "flow": add_then_branch,
        "function_inputs": {"x": orm.Int(3), "y": orm.Int(4)}
    }},
)
print(results)


The `if` statement is ordinary Python; the engine only schedules the branch that is actually taken.  The skipped branch never appears in the WorkGraph.


## Loops (`while`)


In [ ]:
async def keep_doubling(x, y):
    out = add(x, y)
    await wait_for(out)

    while out.result.value < 10:
        inc = add(out.result, 1)
        out = multiply(inc.result, 2)
        await wait_for(out)     # make sure we can read new value

    return {"sum": out.result}

results = run(
    WorkGraphImperativeEngine,
    inputs={"workgraph_data": {
        "name": "add_multiply",
        "flow": keep_doubling,
        "function_inputs": {"x": orm.Int(3), "y": orm.Int(4)}
    }},
)
print(results)


Every iteration adds two more tasks to the WorkGraph.  Because the graph grows dynamically you can observe it expanding live.

> **Performance note** Do not schedule thousands of tiny jobs.


## Waiting for results

`wait_for(task_socket)` suspends the *flow* until the referenced task has reached a terminal state.  This is essential in loops or branches where you need the **value**, not just the future placeholder.  Under the hood it performs an asynchronous poll so your runner remains free to execute other coroutines.

---


## Conclusion

The imperative API combines the **clarity of Python** with the **provenance guarantees of AiiDA**:

* Write workflows as natural coroutines.
* Leverage familiar control structures instead of special DSL constructs.
* Still obtain a fully queryable, shareable WorkGraph.

When you need static analysis or pre‑execution validation drop back to the declarative zones—both styles interoperate because they share the same engine under the hood.
